In [1]:
"""
18092022
Бруцкий-Стемпковский

v.02 Частотный анализ текста.

"""

'\n18092022\nБруцкий-Стемпковский\n\nv.02 Частотный анализ текста.\n\n'

В качестве текста выбрано введение к первому тому "Капитала" К. Маркса. Размер исходного текста превышает 60 000 символов. Написан на русском языке, встречаются слова на латинице. Содержит различные знаки препинания, ссылки в различных скобках, библиотечные шифры, и прочее.

В качестве вспомогательного материала использовал материал из интернета:
1. https://habr.com/ru/post/517410/
2. https://grishaev.me/2012/10/05/1/
3. https://tirinox.ru/levenstein-python/
4. https://pymorphy2.readthedocs.io/en/stable/
5. http://dev.kmint21.info/posts/python-summa/

Импортируем необходимые библиотеки.

In [8]:
import string
import nltk
nltk.download()
import numpy as np
import pandas as pd
import pymorphy2
import time
from functools import lru_cache

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Откроем файл, прочтём его содержимое в переменную text, которая будет содержать текст, закроем файл

In [9]:
file_obj = open("text.txt")
text = file_obj.read()
file_obj.close()

len(text)

64334

Сперва пройдём весь текст и избавимся от большинства знаков препинания, цифр, скобок, знаков табуляции и новой строки. Но так как нам требуется разделить текст на предложения, не будем убирать знаки окончания предложения (точки, знак вопроса, восклицательный знак).

In [10]:
spec_symbol = string.punctuation
spec_symbol = spec_symbol.replace(".", "")
spec_symbol = spec_symbol.replace("!", "")
spec_symbol = spec_symbol.replace("?", "")
spec_symbol += string.digits
spec_symbol += "\n\t\'\"”“–"
symbols_of_end_sentence = ".!?"

spec_symbol

'"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~0123456789\n\t\'"”“–'

Удалим все спецсимволы.

In [11]:
for symbol in spec_symbol:
    text = text.replace(symbol, "\x20")
text = text.replace("...", "\x20")
text = text.lower()

len(text)

64332

Выполним токенизацию текста на предложения с помощью библиотеки NLTK.

In [14]:
text_list = nltk.tokenize.sent_tokenize(text)
number_of_sentence = len(text_list)
list_of_words_in_sentence = []
for _ in text_list:
    list_of_words_in_sentence.append(len(_.split()))
number_of_sentence, len(list_of_words_in_sentence)

(406, 406)

Удалим символы конца строки из текста, выполним токенизацию по словам.

In [17]:
text = text.replace(".", "")
text = text.replace("!", "")
text = text.replace("?", "")
text_list = nltk.tokenize.word_tokenize(text)
len(text_list)

8412

Удалим короткие слова (менее 4-х символов).

In [18]:
text_list = [_ for _ in text_list if len(_) > 3]

len(text_list)

6000

Загрузим список стоп-слов

In [19]:
file_obj = open("stop_words.txt", "r")
russian_stopwords = file_obj.read().split()
file_obj.close()

len(russian_stopwords)

262

Удалим стоп слова

In [20]:
text_list = [_ for _ in text_list if not _ in russian_stopwords]

len(text_list)

5115